In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import pandas as pd
from tqdm import tqdm
import abcd.data.VARS as VARS
from abcd.local.paths import core_path, output_path
from abcd.data.read_data import add_event_vars, filter_subjects, add_subject_sex, get_subjects_events_sf, filter_df_by_visits, add_subject_vars, filter_events, get_subjects_events_visits
from abcd.data.define_splits import SITES, save_restore_visit_splits

In [3]:
subjects_df, events_df = get_subjects_events_sf()
print("There are {} visits and {} subjects".format(len(events_df), len(subjects_df)))   

There are 19327 visits and 9749 subjects


In [4]:
visits = ['baseline_year_1_arm_1']#, '2_year_follow_up_y_arm_1']
subjects_df, events_df = filter_df_by_visits(subjects_df, events_df, visits = visits)
print("There are {} visits and {} subjects".format(len(events_df), len(subjects_df)))  


100%|██████████| 9749/9749 [00:11<00:00, 856.31it/s] 


There are 9377 visits and 9377 subjects


In [5]:
# Adding per-subject NIH Neurocognition scores
subjects_df_nih = add_subject_vars(subjects_df, VARS.NIH_PATH, vars=list(VARS.NIH_TESTS_uncorrected.keys()), leave_first=True)
print("There are {} subjects".format(len(subjects_df_nih)))

c:\users\camgonza\desktop\abcd5\abcd\utils\io.py:25: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=sep, usecols=cols)
100%|██████████| 9377/9377 [00:28<00:00, 329.18it/s]

There are 9233 subjects


In [6]:
# Adding per-subject household income information
subjects_df_income = add_subject_vars(subjects_df, VARS.DEMO_PATH, vars=["demo_comb_income_v2"], leave_first=True)
subjects_df_income = subjects_df_income[subjects_df_income['demo_comb_income_v2'].isin(VARS.VALUES['demo_comb_income_v2'].keys())] # Filtering our 777 and 999
print("There are {} subjects".format(len(subjects_df_income)))

100%|██████████| 9377/9377 [00:34<00:00, 268.68it/s]


There are 8670 subjects


In [7]:
# Adding per-visit CBCL scores
events_df_cbcl = add_event_vars(events_df, VARS.CBCL_PATH, vars=list(VARS.CBCL_SCORES_t.keys()))
events_df_cbcl = events_df_cbcl.dropna() 
subjects_df_cbcl, events_df_cbcl = filter_df_by_visits(subjects_df, events_df_cbcl, visits=visits)
print("There are {} visits and {} subjects".format(len(events_df_cbcl), len(subjects_df_cbcl)))   

100%|██████████| 9373/9373 [00:08<00:00, 1156.73it/s]


There are 9373 visits and 9373 subjects


In [8]:
# Adding per-visit sleep information
events_df_sleep = add_event_vars(events_df, VARS.SLEEP_PATHS['sleepdisturb1_p'], vars=['sleepdisturb1_p', 'sleepdisturb2_p'])
events_df_sleep = events_df_sleep.dropna() 
subjects_df_sleep, events_df_sleep = filter_df_by_visits(subjects_df, events_df_sleep, visits=visits)
print("There are {} visits and {} subjects".format(len(events_df_sleep), len(subjects_df_sleep)))   

100%|██████████| 9376/9376 [00:07<00:00, 1242.65it/s]


There are 9376 visits and 9376 subjects


In [5]:
# All together
subjects_df = add_subject_vars(subjects_df, VARS.NIH_PATH, vars=list(VARS.NIH_TESTS_uncorrected.keys()), leave_first=True)
subjects_df = add_subject_vars(subjects_df, VARS.DEMO_PATH, vars=["demo_comb_income_v2"], leave_first=True)
subjects_df = subjects_df[subjects_df['demo_comb_income_v2'].isin(VARS.VALUES['demo_comb_income_v2'].keys())] # Filtering our 777 and 999
events_df = filter_events(subjects_df, events_df)
print("There are {} visits and {} subjects".format(len(events_df), len(subjects_df))) 
events_df = add_event_vars(events_df, VARS.CBCL_PATH, vars=list(VARS.CBCL_SCORES_t.keys()))
events_df = events_df.dropna() 
subjects_df, events_df = filter_df_by_visits(subjects_df, events_df, visits=visits)
print("There are {} visits and {} subjects".format(len(events_df), len(subjects_df))) 
events_df = add_event_vars(events_df, VARS.SLEEP_PATHS['sleepdisturb1_p'], vars=['sleepdisturb1_p', 'sleepdisturb2_p'])
events_df = events_df.dropna() 
subjects_df, events_df = filter_df_by_visits(subjects_df, events_df, visits=visits)
print("There are {} visits and {} subjects".format(len(events_df), len(subjects_df))) 

c:\users\camgonza\desktop\abcd5\abcd\utils\io.py:25: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=sep, usecols=cols)
100%|██████████| 9233/9233 [00:34<00:00, 264.09it/s]


There are 8537 visits and 8537 subjects


100%|██████████| 8535/8535 [00:17<00:00, 490.24it/s]


There are 8535 visits and 8535 subjects


100%|██████████| 8535/8535 [00:13<00:00, 635.24it/s]


There are 8535 visits and 8535 subjects


In [10]:
k=5
visits = ['baseline_year_1_arm_1', '2_year_follow_up_y_arm_1', '4_year_follow_up_y_arm_1']
subjects_df, events_df = get_subjects_events_visits(visits)
splits = save_restore_visit_splits(visits=visits, k=k)

100%|██████████| 9749/9749 [00:11<00:00, 885.87it/s] 
c:\users\camgonza\desktop\abcd5\abcd\utils\io.py:25: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=sep, usecols=cols)
100%|██████████| 21/21 [00:00<00:00, 64.98it/s]


In [11]:
# Plot number of subjects per split
import pygal
from abcd.plotting.pygal.rendering import display_html
from abcd.plotting.pygal.colors import CAT_COLORS

custom_style = pygal.style.Style(
    colors=tuple([CAT_COLORS['split'][str(split_ix)] for split_ix in range(5)])
    #,background='transparent'
    )

bar_chart = pygal.Bar(x_label_rotation=45, style=custom_style)
bar_chart.title = '# subjects per split'
bar_chart.x_labels = [x.replace("site", "site ") for x in SITES]
for split_ix in range(k):
    bar_chart.add(str(split_ix), [len(splits[site][str(split_ix)]) for site in SITES])
display_html(bar_chart)